# Preparing

In [1]:
import os
import pandas as pd

import pickle

# Configuration

In [2]:
model_patch = {}

# Target dataset (which is used for the true classification)
key_dataset = 'coffee.xlsx'
#dict_name = 'coffee_dict.xlsx'
non_coffee = 'content/stopwords/non-cofe.xlsx'

# *** Datasets ***
stream_path = 'content/stream/stream/'

# *** Classifieng models ***
model_patch["rfc"] = "content/models/random_forest_classifier.pickle"
model_patch["sgd"] = "content/models/sgd_classifier.pickle"

# *** For saving ***
save_patch = 'content/analyse/'
trash_path = 'content/analyse/'

# Dataset structure - used for the source datasets fields
req_str = 'text'

brand_weight_multiplicator = 64
non_category_weight_multiplicator = 1

# Load data

In [3]:
# *** Loadingf of dictionaries with brands and support words ***
stream_df = pd.DataFrame()

for entry in os.listdir(stream_path):
    if os.path.isfile(os.path.join(stream_path, entry)):
        
        tmp_stream = pd.read_csv(
            stream_path + entry, 
            encoding = 'utf8', 
            header=None,
            sep='\\n',
            engine='python'
        )
        
        stream_df = pd.concat(
            [stream_df, tmp_stream], 
            sort=False
        ).sample(frac = 1).reset_index(drop=True)

print(stream_df.shape)

(9472392, 1)


# Preprocess data analysing

In [4]:
print(stream_df.head(5))

                                                   0
0                             скандал домогательства
1                           крыша со светом коровник
2                           кинолог для щенка москва
3      деревянный стол с салфетками фон для фотошопа
4  непредставление финансов субъектом рф для реал...


In [5]:
# drop NaN
trash_df = pd.DataFrame()
trash_df = stream_df[stream_df[0].isna()]
stream_df = stream_df.dropna()
trash_df

,0
649277,NaN
3002827,NaN
3455674,NaN
7109658,NaN
8133788,NaN


In [6]:
trash_df.shape

(5, 1)

In [7]:
stream_df.describe(include='all')  

,0
count,9472387
unique,9371543
top,-
freq,16


In [8]:
# By the customer info, the numbers type are useful if a request includes more than one word 
stream_df = stream_df.astype(str)

In [9]:
count = stream_df[0].str.split().str.len()

trash_df = pd.concat(
            [
                trash_df, 
                stream_df[~(count!=1).copy()]
            ], 
            sort=False
        )

stream_df = stream_df.drop(stream_df[~(count!=1)].copy().index)

In [10]:
trash_df.head(-1)

,0
649277,NaN
3002827,NaN
3455674,NaN
7109658,NaN
8133788,NaN
...,...
9472359,food+wrap+for+food
9472367,ttps
9472369,java+collectors+to+map
9472375,4000667


In [11]:
trash_df.shape

(1786933, 1)

In [12]:
stream_df.head(-1)

,0
0,скандал домогательства
1,крыша со светом коровник
2,кинолог для щенка москва
3,деревянный стол с салфетками фон для фотошопа
4,непредставление финансов субъектом рф для реал...
...,...
9472386,создать эдектронную почту яндекс
9472387,азбука дорожного движения для детей
9472388,Boreyko Rem
9472389,что такое кинематические параметры


In [13]:
# Too long is also strange
count = stream_df[0].str.split().str.len()

trash_df = pd.concat(
            [
                trash_df.copy(), 
                stream_df[~(count<12).copy()]
            ], 
            sort=False
        )

stream_df = stream_df.drop(stream_df[~(count<12)].copy().index)

In [14]:
trash_df.shape, stream_df.shape

((2015090, 1), (7457302, 1))

# Get Prediction

In [15]:
model = pickle.load(open(model_patch["sgd"], 'rb'))

In [16]:
model

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=1000,
                                 min_df=5, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 SGDClassifier(alpha=0.0001, average=False,
                               class_weight='balanced', early_stopping=False,
                               epsilon=0.1, eta0=0.0, fit_intercept=True,
                               l1_ratio=0.15, learning_rate='optimal',
                   

In [17]:
predicted_sgd_bin = model.predict_proba(stream_df[0])
predicted_sgd_bin

array([[0.33789488, 0.66210512],
       [0.44319352, 0.55680648],
       [0.82568877, 0.17431123],
       ...,
       [0.460032  , 0.539968  ],
       [0.33789488, 0.66210512],
       [0.20750846, 0.79249154]])

In [18]:
len(predicted_sgd_bin), stream_df[0].shape

(7457302, (7457302,))

In [19]:
tmp2 = pd.DataFrame(predicted_sgd_bin)
tmp2.head(-1)

,0,1
0,0.337895,0.662105
1,0.443194,0.556806
2,0.825689,0.174311
3,0.756058,0.243942
4,0.790770,0.209230
...,...,...
7457296,0.337895,0.662105
7457297,0.845965,0.154035
7457298,0.337895,0.662105
7457299,0.460032,0.539968


# Get Results Probabilities

In [20]:
result = pd.DataFrame()

stream_df = stream_df.rename(columns={0: "query"})

result = pd.concat([
    stream_df["query"].copy().reset_index(drop=True), 
    pd.DataFrame(predicted_sgd_bin)], axis=1) 
result

,query,0,1
0,скандал домогательства,0.337895,0.662105
1,крыша со светом коровник,0.443194,0.556806
2,кинолог для щенка москва,0.825689,0.174311
3,деревянный стол с салфетками фон для фотошопа,0.756058,0.243942
4,непредставление финансов субъектом рф для реал...,0.790770,0.209230
...,...,...,...
7457297,азбука дорожного движения для детей,0.845965,0.154035
7457298,Boreyko Rem,0.337895,0.662105
7457299,что такое кинематические параметры,0.460032,0.539968
7457300,передача территории монголии,0.337895,0.662105


In [21]:
threshold = [1.0, 0.9, 0.8, 0.7]
for i in threshold:
    good = predicted_sgd_bin[:, 1]
    predicted_good = good > i
    result["P=" + str(i)] = predicted_good

In [22]:
result

,query,0,1,P=1.0,P=0.9,P=0.8,P=0.7
0,скандал домогательства,0.337895,0.662105,False,False,False,False
1,крыша со светом коровник,0.443194,0.556806,False,False,False,False
2,кинолог для щенка москва,0.825689,0.174311,False,False,False,False
3,деревянный стол с салфетками фон для фотошопа,0.756058,0.243942,False,False,False,False
4,непредставление финансов субъектом рф для реал...,0.790770,0.209230,False,False,False,False
...,...,...,...,...,...,...,...
7457297,азбука дорожного движения для детей,0.845965,0.154035,False,False,False,False
7457298,Boreyko Rem,0.337895,0.662105,False,False,False,False
7457299,что такое кинематические параметры,0.460032,0.539968,False,False,False,False
7457300,передача территории монголии,0.337895,0.662105,False,False,False,False


In [23]:
result[result[1] > 0.7]

,query,0,1,P=1.0,P=0.9,P=0.8,P=0.7
22,фильмы похожие на помни меня,0.207508,0.792492,False,False,False,True
33,схема температурного 3-х контактного датчика н...,0.207508,0.792492,False,False,False,True
35,заявление в прокуратуру об аудиторской проверк...,0.207508,0.792492,False,False,False,True
50,полипропиленовые трубы на 32,0.207508,0.792492,False,False,False,True
73,решение задач на противоположное движение,0.207508,0.792492,False,False,False,True
...,...,...,...,...,...,...,...
7457212,tears of themis на пк,0.207508,0.792492,False,False,False,True
7457219,убийство на ферме уайнхаус смотреть,0.207508,0.792492,False,False,False,True
7457264,Ограничение цен на сахар,0.207508,0.792492,False,False,False,True
7457291,6 серия на караоке,0.207508,0.792492,False,False,False,True


In [24]:
print("P=1")
print(result["P=1.0"].value_counts())
print("")
print("P=0.9")
print(result["P=0.9"].value_counts())
print("")
print("P=0.8")
print(result["P=0.8"].value_counts())
print("")
print("P=0.7")
print(result["P=0.7"].value_counts())

P=1
False    7457302
Name: P=1.0, dtype: int64

P=0.9
False    7452001
True        5301
Name: P=0.9, dtype: int64

P=0.8
False    7445586
True       11716
Name: P=0.8, dtype: int64

P=0.7
False    7079277
True      378025
Name: P=0.7, dtype: int64


# Save files for analysing

In [27]:
result.shape

(7457302, 7)

In [33]:
i = int(result.shape[0] / 4)
result[:i].to_csv(save_patch + 'result1.gz', compression='gzip')
result[i:(2*i)].to_csv(save_patch + 'result2.gz', compression='gzip')
result[(2*i):(3*i)].to_csv(save_patch + 'result3.gz', compression='gzip')
result[(3*i):].to_csv(save_patch + 'result4.gz', compression='gzip')

In [25]:
result.to_csv(save_patch + 'result.gz', compression='gzip')

In [26]:
trash_df.to_csv(trash_path + 'trash.gz', compression='gzip')